# Part 4: Agentic RAG with a Custom MCP Server

**[EN]** This notebook provides a unified setup guide for connecting Claude Desktop to your custom MCP server. It supports running the server both locally (on Windows/Mac/Linux) and remotely.
**[KR]**  이 노트북은 Claude Desktop을 자체 제작한 MCP 서버에 연결하기 위한 통합 설치 안내서입니다. 로컬(Windows/Mac/Linux) 및 원격 환경 모두에서 서버를 실행하는 시나리오를 지원합니다.

## Step 1: Prerequisites

**[EN]** Ensure you have Docker, docker-compose, and Claude Desktop installed on your local machine.<br>
**[KR]** 로컬 머신에 Docker, docker-compose와 Claude Desktop이 설치되어 있는지 확인해주세요.

## Step 2: Run Server (if necessary) and Generate Configuration

**[EN]** Run the single cell below. It will guide you through the entire process: running the server locally if needed, and generating the correct configuration for your specific setup (Local/Remote).<br>
**[KR]** 아래의 단일 셀을 실행하세요. 이 셀이 전체 과정을 안내합니다: 필요시 서버를 로컬에서 실행하고, 현재 설정(로컬/원격)에 맞는 올바른 설정을 생성합니다.

In [ ]:
import os
import json
import platform
import subprocess

server_location = ""
while server_location not in ["local", "remote"]:
    server_location = input("Is the MCP server running on your LOCAL machine or a REMOTE machine? (local/remote): ").lower()

REMOTE_SERVER_IP = ""

if server_location == 'local':
    print("--- Setting up local server using docker-compose ---")
    
    if not os.path.exists('elastic.env'):
        print("⚠️ Error: 'elastic.env' file not found. Please create it before running.")
    else:
        print("Starting services with 'docker-compose up -d --build'... This might take a while.")
        result = subprocess.run("docker-compose up -d --build", shell=True, capture_output=True, text=True, check=False)
        if result.returncode == 0:
            print("\n✅ Local server and proxy are up and running!")
        else:
            print("\n❌ Error starting services with docker-compose:")
            print(result.stderr)

    os_type = platform.system()
    if os_type == "Windows" or os_type == "Darwin":
        REMOTE_SERVER_IP = "localhost"
        print(f"\nDetected {os_type}, using '{REMOTE_SERVER_IP}' for local connection.")
    else: # Assuming Linux
        REMOTE_SERVER_IP = "localhost"
        print(f"\nDetected {os_type}, using '{REMOTE_SERVER_IP}' for local connection.")
else: # remote
    print("--- Generating configuration for REMOTE server ---")
    print("Please ensure your services are running on the remote machine (e.g., with 'docker-compose up -d')")
    REMOTE_SERVER_IP = input("Please enter the public IP address of your remote server: ")

# For local connections, we connect directly to the proxy's exposed port.
# For remote connections, the user provides the public IP.
config = {
    "mcpServers": {
        "colpali-search-server": {
            "type": "websocket",
            "url": f"ws://{REMOTE_SERVER_IP}:8000"
        }
    }
}

print("\n✅ Configuration generated successfully!")
print("👇 Copy the JSON below and paste it into your local claude_desktop_config.json file:")
print("-" * 70)
print(json.dumps(config, indent=4))
print("-" * 70)

## Step 3: Final Steps

**[EN]** 
1. **Open your Claude Desktop configuration file.**
2. **Paste the configuration.**
3. **Restart Claude Desktop.**

**[KR]** 
1. **Claude Desktop 설정 파일을 엽니다.**
2. **설정을 붙여넣습니다.**
3. **Claude Desktop을 재시작하세요.**